# LOADS

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd "/content/gdrive/Shareddrives/NLP_Lab_3"

/content/gdrive/Shareddrives/NLP_Lab_3


In [ ]:
!pip install datasets evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00


In [ ]:
! pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-juoe4k0w
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-juoe4k0w
  Resolved https://github.com/huggingface/transformers to commit bdb9106f247fca48a71eb384be25dbbd29b065a8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9128517 sha256=6f70090baacdc7e3b8ca5716cd33e7fc4a9680a967aa88bd8bf1132482093738
  Stored in directory: /tmp/pip-ephem-wheel-cache-5yq42j3u/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninstalling transformers-4.41.0:
      Successfully uninstalled transformers-4.41.0


In [ ]:
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

We will work with the goemotions dataset for sentiment analysis task:

https://github.com/google-research/google-research/tree/master/goemotions

In [ ]:
import pandas as pd

df = pd.read_csv("data/train.tsv", sep = "\t", names = ["text", "labels", "id"])
df[:20]

,text,labels,id
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,26,edvnz26
6,Yes I heard abt the f bombs! That has to be wh...,15,ee3b6wu
7,We need more boards and to create a bit more s...,"8,20",ef4qmod
8,Damn youtube and outrage drama is super lucrat...,0,ed8wbdn
9,It might be linked to the trust factor of your...,27,eczgv1o


For a single-label classification, we select the first class for every example and save the result into a comma-separated CSV file

For a multi-label classification task: the dataset shall be converted into the JSON format, where every example goes in a separate line

In [ ]:
import pandas as pd
import json


training_labels = set()
neutral_class = "27"

for set_type in ["train", "dev", "test"]:
    df = pd.read_csv("data/%s.tsv" % (set_type), sep = "\t", names = ["text", "labels", "id"])
    df["single_label"] = df["labels"].apply(lambda x: x.split(",")[0])
    df[["text", "single_label"]].to_csv("data/%s_single_label.csv" % (set_type), index=False)

    with open("data/%s.json" % (set_type), "w") as fout:
        for nrow,row in df.iterrows():
            example = {}
            example["text"] = row["text"]
            example["labels"] = row["labels"].split(",")
            if set_type!="train":
                example["labels"] = [label if label in training_labels else neutral_class for label in example["labels"]]
            json.dump(example, fout)
            fout.write("\n")
            if set_type == "train":
                training_labels.update(example["labels"])

Download run_classification.py into your folder with this notebook from:

https://github.com/huggingface/transformers/tree/main/examples/pytorch/text-classification

Run training using GPU (Runtime -> Change runtime type -> T4 GPU)

# 3 EPOCHS - 32 BATCH SIZE - 128 MAX SEQ LENGHT - 2e-5 LEARNING RATE

In [ ]:
# For single-label classifiction, we provide .csv files
#! python run_classification.py --model_name_or_path  bert-large-uncased --train_file "data/train_single_label.csv" --validation_file "data/dev_single_label.csv" --test_file "data/test_single_label.csv" --shuffle_train_dataset --metric_name f1 --text_column_name "text" --label_column_name "single_label" --do_train --do_eval --do_predict --max_seq_length 128 --per_device_train_batch_size 32 --learning_rate 2e-5 --num_train_epochs 5 --output_dir bert_large_uncased_goemotions_single_label

In [ ]:
# For multi-label classifiction, we provide .json files
!python run_classification.py --model_name_or_path  bert-large-uncased --train_file "data/train.json" --validation_file "data/dev.json" --test_file "data/test.json" --shuffle_train_dataset --metric_name f1 --text_column_name "text" --label_column_name "labels" --do_train --do_eval --do_predict --max_seq_length 128 --per_device_train_batch_size 32 --learning_rate 2e-5 --num_train_epochs 3 --output_dir bert_large_uncased_goemotions

2024-05-19 15:33:54.455511: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 15:33:54.455577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 15:33:54.456991: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-19 15:33:54.464224: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 15:33:55.755730: W tensorflow/comp

In [ ]:
# Evaluation for single-label classification
'''
from sklearn.metrics import classification_report
import pandas as pd
import re

df_test = pd.read_csv("data/test_single_label.csv")
df_test_preds = pd.read_csv("bert_large_uncased_goemotions_single_label/predict_results.txt", sep="\t")
true_labels = df_test["single_label"].tolist()
pred_labels = df_test_preds["prediction"].tolist()
print(classification_report(true_labels, pred_labels))
'''

'\nfrom sklearn.metrics import classification_report\nimport pandas as pd\nimport re\n\ndf_test = pd.read_csv("data/test_single_label.csv")\ndf_test_preds = pd.read_csv("bert_large_uncased_goemotions_single_label/predict_results.txt", sep="\t")\ntrue_labels = df_test["single_label"].tolist()\npred_labels = df_test_preds["prediction"].tolist()\nprint(classification_report(true_labels, pred_labels))\n'

In [ ]:
# Evaluation for multi-label classification

from sklearn.metrics import classification_report
import pandas as pd
import json
import re

num_classes = 28
true_labels = []
pred_labels = []
with open("data/test.json", "r") as fin:
    for line in fin:
        multi_hot_vector = [0]*num_classes
        labels = json.loads(line)["labels"]
        for label in labels:
            multi_hot_vector[int(label)] = 1
        true_labels.append(multi_hot_vector.copy())

df_test_preds = pd.read_csv("bert_large_uncased_goemotions/predict_results.txt", sep="\t")
pred_labels_as_str = df_test_preds["prediction"].tolist()
for line in pred_labels_as_str:
    multi_hot_vector = [0]*num_classes
    labels = json.loads(re.sub("'", "\"", line))
    for label in labels:
        multi_hot_vector[int(label)] = 1
    pred_labels.append(multi_hot_vector.copy())
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.70      0.73      0.72       504
           1       0.79      0.85      0.82       264
           2       0.62      0.43      0.51       198
           3       0.49      0.20      0.28       320
           4       0.59      0.32      0.41       351
           5       0.58      0.41      0.48       135
           6       0.57      0.33      0.42       153
           7       0.55      0.49      0.52       284
           8       0.66      0.35      0.46        83
           9       0.58      0.15      0.23       151
          10       0.47      0.34      0.40       267
          11       0.65      0.33      0.43       123
          12       0.77      0.27      0.40        37
          13       0.69      0.30      0.42       103
          14       0.70      0.65      0.68        78
          15       0.94      0.90      0.92       352
          16       0.00      0.00      0.00         6
          17       0.72    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run your fine-tuned model for a single inference:

In [ ]:
import torch
from transformers import pipeline

label_names = open("data/emotions.txt", "r").read().splitlines()
print(label_names)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
pipe = pipeline("text-classification", model="bert_large_uncased_goemotions", return_all_scores=True, device=-1 if device.type=="cpu" else 0)
#pipe = pipeline("text-classification", model="bhadresh-savani/bert-base-uncased-emotion", return_all_scores=True, device=-1 if device.type == "cpu" else 0)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
cuda


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# You may tune this threshold manually or using the development set.
# It shows how confident the model can be in the class to take this class for the outcome.
# A higher threshold may cause an increase in precision but a drop in recall.
# Lower values might get higher recall and lower precision. Some trade-off that increases the overall f1-score can be found.
threshold = 0.5

#text = "It's great that you're a recovering addict, that's cool. Have you ever tried DMT?"
text = "Lol! But I love your last name though. XD"
all_classes_scored = pipe(text)
print(all_classes_scored)

predictions = []
for pred in all_classes_scored[0]:
    if pred["score"] > threshold:
        predictions.append((pred["label"], pred["score"]))
print(predictions)
for pred in predictions:
    print(label_names[int(pred[0])])
    #print(pred[0])

[[{'label': '0', 'score': 0.03638361021876335}, {'label': '1', 'score': 0.7485623955726624}, {'label': '10', 'score': 0.013902966864407063}, {'label': '11', 'score': 0.005694305524230003}, {'label': '12', 'score': 0.0047791581600904465}, {'label': '13', 'score': 0.008352021686732769}, {'label': '14', 'score': 0.007438547443598509}, {'label': '15', 'score': 0.010110200382769108}, {'label': '16', 'score': 0.0027108625508844852}, {'label': '17', 'score': 0.04487846791744232}, {'label': '18', 'score': 0.7560065388679504}, {'label': '19', 'score': 0.002816347870975733}, {'label': '2', 'score': 0.008922049775719643}, {'label': '20', 'score': 0.011926557868719101}, {'label': '21', 'score': 0.004613838624209166}, {'label': '22', 'score': 0.008905000984668732}, {'label': '23', 'score': 0.005132374819368124}, {'label': '24', 'score': 0.0031983035150915384}, {'label': '25', 'score': 0.010042086243629456}, {'label': '26', 'score': 0.009069588035345078}, {'label': '27', 'score': 0.01455884799361229

# 1 EPOCH - 32 BATCH SIZE - 128 MAX SEQ LENGHT - 2e-5 LEARNING RATE

In [ ]:
# For multi-label classifiction, we provide .json files
!python run_classification.py --model_name_or_path  bert-large-uncased --train_file "data/train.json" --validation_file "data/dev.json" --test_file "data/test.json" --shuffle_train_dataset --metric_name f1 --text_column_name "text" --label_column_name "labels" --do_train --do_eval --do_predict --max_seq_length 128 --per_device_train_batch_size 32 --learning_rate 2e-5 --num_train_epochs 1 --output_dir bert_large_uncased_goemotions_1_epoch

2024-05-23 10:49:18.969710: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 10:49:18.969764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 10:49:18.973698: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-23 10:49:18.981028: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 10:49:20.124333: W tensorflow/comp

In [ ]:
# Evaluation for multi-label classification

from sklearn.metrics import classification_report
import pandas as pd
import json
import re

num_classes = 28
true_labels = []
pred_labels = []
with open("data/test.json", "r") as fin:
    for line in fin:
        multi_hot_vector = [0]*num_classes
        labels = json.loads(line)["labels"]
        for label in labels:
            multi_hot_vector[int(label)] = 1
        true_labels.append(multi_hot_vector.copy())

df_test_preds = pd.read_csv("bert_large_uncased_goemotions_1_epoch/predict_results.txt", sep="\t")
pred_labels_as_str = df_test_preds["prediction"].tolist()
for line in pred_labels_as_str:
    multi_hot_vector = [0]*num_classes
    labels = json.loads(re.sub("'", "\"", line))
    for label in labels:
        multi_hot_vector[int(label)] = 1
    pred_labels.append(multi_hot_vector.copy())
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.72      0.61      0.66       504
           1       0.80      0.85      0.82       264
           2       0.69      0.15      0.24       198
           3       0.00      0.00      0.00       320
           4       0.74      0.14      0.24       351
           5       0.00      0.00      0.00       135
           6       0.46      0.04      0.07       153
           7       0.56      0.35      0.43       284
           8       0.00      0.00      0.00        83
           9       0.00      0.00      0.00       151
          10       0.68      0.06      0.12       267
          11       0.00      0.00      0.00       123
          12       0.00      0.00      0.00        37
          13       0.00      0.00      0.00       103
          14       1.00      0.05      0.10        78
          15       0.95      0.89      0.92       352
          16       0.00      0.00      0.00         6
          17       0.74    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 1 EPOCH - 16 BATCH SIZE - 128 MAX SEQ LENGHT - 3e-5 LEARNING RATE - BETTER

In [ ]:
#i get the CUDA out of memory error typically occurs when your GPU does not have enough memory to handle the batch size you have specified -> batch size 64 cannot
!python run_classification.py --model_name_or_path  bert-large-uncased --train_file "data/train.json" --validation_file "data/dev.json" --test_file "data/test.json" --shuffle_train_dataset --metric_name f1 --text_column_name "text" --label_column_name "labels" --do_train --do_eval --do_predict --max_seq_length 128 --per_device_train_batch_size 16 --learning_rate 3e-5 --num_train_epochs 1 --output_dir bert_large_uncased_goemotions_1_epoch_cambioos

2024-05-25 18:47:20.366143: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 18:47:20.366220: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 18:47:20.451048: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-25 18:47:20.463196: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-25 18:47:22.228609: W tensorflow/compiler/tf2

In [ ]:
# Evaluation for multi-label classification

from sklearn.metrics import classification_report
import pandas as pd
import json
import re

num_classes = 28
true_labels = []
pred_labels = []
with open("data/test.json", "r") as fin:
    for line in fin:
        multi_hot_vector = [0]*num_classes
        labels = json.loads(line)["labels"]
        for label in labels:
            multi_hot_vector[int(label)] = 1
        true_labels.append(multi_hot_vector.copy())

df_test_preds = pd.read_csv("bert_large_uncased_goemotions_1_epoch_cambioos/predict_results.txt", sep="\t")
pred_labels_as_str = df_test_preds["prediction"].tolist()
for line in pred_labels_as_str:
    multi_hot_vector = [0]*num_classes
    labels = json.loads(re.sub("'", "\"", line))
    for label in labels:
        multi_hot_vector[int(label)] = 1
    pred_labels.append(multi_hot_vector.copy())
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.70      0.68      0.69       504
           1       0.79      0.88      0.83       264
           2       0.66      0.32      0.43       198
           3       0.58      0.11      0.18       320
           4       0.70      0.22      0.34       351
           5       0.70      0.17      0.27       135
           6       0.65      0.26      0.37       153
           7       0.59      0.39      0.47       284
           8       0.65      0.31      0.42        83
           9       0.00      0.00      0.00       151
          10       0.52      0.18      0.26       267
          11       0.67      0.24      0.35       123
          12       1.00      0.05      0.10        37
          13       0.85      0.21      0.34       103
          14       0.65      0.65      0.65        78
          15       0.96      0.87      0.91       352
          16       0.00      0.00      0.00         6
          17       0.71    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 1 EPOCH - 16 BATCH SIZE - 64 MAX SEQ LENGHT - 3e-5 LEARNING RATE - SIMILAR

In [ ]:
# For multi-label classifiction, we provide .json files
!python run_classification.py --model_name_or_path  bert-large-uncased --train_file "data/train.json" --validation_file "data/dev.json" --test_file "data/test.json" --shuffle_train_dataset --metric_name f1 --text_column_name "text" --label_column_name "labels" --do_train --do_eval --do_predict --max_seq_length 64 --per_device_train_batch_size 16 --learning_rate 3e-5 --num_train_epochs 1 --output_dir bert_large_uncased_goemotions_1_epoch_cambios_ult

2024-05-24 09:29:42.983302: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 09:29:42.983352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 09:29:43.091462: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-24 09:29:43.118963: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-24 09:29:44.168096: W tensorflow/comp

In [ ]:
# Evaluation for multi-label classification

from sklearn.metrics import classification_report
import pandas as pd
import json
import re

num_classes = 28
true_labels = []
pred_labels = []
with open("data/test.json", "r") as fin:
    for line in fin:
        multi_hot_vector = [0]*num_classes
        labels = json.loads(line)["labels"]
        for label in labels:
            multi_hot_vector[int(label)] = 1
        true_labels.append(multi_hot_vector.copy())

df_test_preds = pd.read_csv("bert_large_uncased_goemotions_1_epoch_cambios_ult/predict_results.txt", sep="\t")
pred_labels_as_str = df_test_preds["prediction"].tolist()
for line in pred_labels_as_str:
    multi_hot_vector = [0]*num_classes
    labels = json.loads(re.sub("'", "\"", line))
    for label in labels:
        multi_hot_vector[int(label)] = 1
    pred_labels.append(multi_hot_vector.copy())
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.72      0.67      0.69       504
           1       0.77      0.90      0.83       264
           2       0.71      0.36      0.48       198
           3       0.75      0.09      0.17       320
           4       0.66      0.20      0.31       351
           5       0.74      0.19      0.30       135
           6       0.60      0.27      0.38       153
           7       0.58      0.44      0.50       284
           8       0.73      0.27      0.39        83
           9       0.00      0.00      0.00       151
          10       0.54      0.16      0.25       267
          11       0.89      0.07      0.12       123
          12       0.00      0.00      0.00        37
          13       0.83      0.19      0.31       103
          14       0.70      0.62      0.65        78
          15       0.96      0.89      0.92       352
          16       0.00      0.00      0.00         6
          17       0.70    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 3 EPOCHS - 16 BATCH SIZE - 128 MAX SEQ LENGHT - 3e-5 LEARNING RATE - TODO

In [ ]:
#i get the CUDA out of memory error typically occurs when your GPU does not have enough memory to handle the batch size you have specified -> batch size 64 cannot
!python run_classification.py --model_name_or_path  bert-large-uncased --train_file "data/train.json" --validation_file "data/dev.json" --test_file "data/test.json" --shuffle_train_dataset --metric_name f1 --text_column_name "text" --label_column_name "labels" --do_train --do_eval --do_predict --max_seq_length 128 --per_device_train_batch_size 16 --learning_rate 3e-5 --num_train_epochs 3 --output_dir bert_large_uncased_goemotions_3_epochs_may_be_better

2024-05-26 08:52:20.690278: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 08:52:20.690344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 08:52:20.775391: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-26 08:52:20.794094: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 08:52:22.815738: W tensorflow/compiler/tf2

In [ ]:
# Evaluation for multi-label classification

from sklearn.metrics import classification_report
import pandas as pd
import json
import re

num_classes = 28
true_labels = []
pred_labels = []
with open("data/test.json", "r") as fin:
    for line in fin:
        multi_hot_vector = [0]*num_classes
        labels = json.loads(line)["labels"]
        for label in labels:
            multi_hot_vector[int(label)] = 1
        true_labels.append(multi_hot_vector.copy())

df_test_preds = pd.read_csv("bert_large_uncased_goemotions_3_epochs_may_be_better/predict_results.txt", sep="\t")
pred_labels_as_str = df_test_preds["prediction"].tolist()
for line in pred_labels_as_str:
    multi_hot_vector = [0]*num_classes
    labels = json.loads(re.sub("'", "\"", line))
    for label in labels:
        multi_hot_vector[int(label)] = 1
    pred_labels.append(multi_hot_vector.copy())
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.69      0.72      0.70       504
           1       0.78      0.86      0.82       264
           2       0.58      0.46      0.51       198
           3       0.47      0.26      0.34       320
           4       0.52      0.33      0.40       351
           5       0.52      0.41      0.46       135
           6       0.54      0.39      0.45       153
           7       0.55      0.50      0.52       284
           8       0.67      0.40      0.50        83
           9       0.55      0.23      0.33       151
          10       0.45      0.39      0.42       267
          11       0.57      0.37      0.45       123
          12       0.76      0.35      0.48        37
          13       0.58      0.35      0.44       103
          14       0.71      0.64      0.68        78
          15       0.96      0.89      0.92       352
          16       0.00      0.00      0.00         6
          17       0.69    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
